# Data Clean Up 

In [9]:
# Import modules
import os
import pandas as pd

## Make library data set

In [10]:
# Set parameters
covid_path = "../Resources/Weekly_COVID-19_Cases__Tests__and_Deaths_by_ZIP_Code_pulled_272022.csv"
library_circulation_2019_path = "../Resources/Libraries_-_2019_Circulation_by_Location_pulled_272022.csv"
library_visitors_2019_path = "../Resources/Libraries_-_2019_Visitors_by_Location_pulled_272022.csv"
library_circulation_2020_path = "../Resources/Libraries_-_2020_Circulation_by_Location_pulled_272022.csv"
library_visitors_2020_path = "../Resources/Libraries_-_2020_Visitors_by_Location_pulled_272022.csv"
library_circulation_2021_path = "../Resources/Libraries_-_2021_Circulation_by_Location_pulled_272022.csv"
library_visitors_2021_path = "../Resources/Libraries_-_2021_Visitors_by_Location_pulled_272022.csv"

In [11]:
# Load datasets
covid_df = pd.read_csv(covid_path)
library_circulation_2019_df = pd.read_csv(library_circulation_2019_path, dtype={'ZIP': object})
library_visitors_2019_df = pd.read_csv(library_visitors_2019_path, dtype={'ZIP': object})
library_circulation_2020_df = pd.read_csv(library_circulation_2020_path, dtype={'ZIP': object})
library_visitors_2020_df = pd.read_csv(library_visitors_2020_path, dtype={'ZIP': object})
library_circulation_2021_df = pd.read_csv(library_circulation_2021_path, dtype={'ZIP': object})
library_visitors_2021_df = pd.read_csv(library_visitors_2021_path, dtype={'ZIP': object})

In [12]:
# For library circulation we have data without zip code (online renewal, media download, etc.)
# We want to drop this data since it has no zip code associated with it
library_circulation_2019_df = library_circulation_2019_df.dropna(how='any')
library_circulation_2020_df = library_circulation_2020_df.dropna(how='any')
library_circulation_2021_df = library_circulation_2021_df.dropna(how='any')

In [17]:
# Align zip code column name
covid_df = covid_df.rename(columns={"ZIP Code": "ZIP"})

# Change LOCATION to BRANCH for 2019 data to match 2020 and 2021 format
library_circulation_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})
library_visitors_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})

# Change Location to LOCATION
library_circulation_2021_df = library_circulation_2021_df.rename(columns={"Location": "LOCATION"})

In [14]:
# Add type column
library_circulation_2019_df["TYPE"] = ["Circulation"] * len(library_circulation_2019_df)
library_visitors_2019_df["TYPE"] = ["Visitors"] * len(library_visitors_2019_df)
library_circulation_2020_df["TYPE"] = ["Circulation"] * len(library_circulation_2020_df)
library_visitors_2020_df["TYPE"] = ["Visitors"] * len(library_visitors_2020_df)
library_circulation_2021_df["TYPE"] = ["Circulation"] * len(library_circulation_2021_df)
library_visitors_2021_df["TYPE"] = ["Visitors"] * len(library_visitors_2021_df)

# Add year column
library_circulation_2019_df["YEAR"] = [2019] * len(library_circulation_2019_df)
library_visitors_2019_df["YEAR"] = [2019] * len(library_visitors_2019_df)
library_circulation_2020_df["YEAR"] = [2020] * len(library_circulation_2020_df)
library_visitors_2020_df["YEAR"] = [2020] * len(library_visitors_2020_df)
library_circulation_2021_df["YEAR"] = [2021] * len(library_circulation_2021_df)
library_visitors_2021_df["YEAR"] = [2021] * len(library_visitors_2021_df)

In [15]:
# Merge circulation and visitors together
merged_2019 = pd.merge(library_circulation_2019_df, library_visitors_2019_df, how = "outer")
merged_2020 = pd.merge(library_circulation_2020_df, library_visitors_2020_df, how = "outer")
merged_2021 = pd.merge(library_circulation_2021_df, library_visitors_2021_df, how = "outer")

# Merge all together
merged = pd.merge(merged_2019, merged_2020, how = "outer")
merged = pd.merge(merged, merged_2021, how = "outer")

# Remove erroneous columns
merged = merged.drop(['LOCATION'], axis = 1)

# Remove index column
merged = merged.set_index('BRANCH')
merged.head()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR
BRANCH,,,,,,,,,,,,,,,,,,
Albany Park,3401 W. Foster Ave.,Chicago,60625,8214,7614,8460,7414,7044,7970,9664,9069,7053,8012,6571,5924,93009,Circulation,2019
Altgeld,13281 S. Corliss Ave.,Chicago,60827,378,326,332,270,320,325,354,229,234,377,369,500,4014,Circulation,2019
Archer Heights,5055 S. Archer Ave.,Chicago,60632,5365,5019,5417,5349,4833,4627,5356,4793,4537,5263,4051,3415,58025,Circulation,2019
Austin,5615 W. Race Ave.,Chicago,60644,215,590,706,736,806,983,1134,1236,1193,1361,1286,1082,11328,Circulation,2019
Austin-Irving,6100 W. Irving Park Rd.,Chicago,60634,9678,9764,10845,10541,9763,11442,11634,11063,9379,9932,8567,6492,119100,Circulation,2019


In [16]:
# Export data
merged.to_csv("../Data/merged_library.csv")

## Make COVID data set

In [18]:
# covid_df.head()
# len(covid_df)
covid_df['Year'] = pd.to_datetime(covid_df['Week Start']).dt.year
covid_df['Month'] = pd.to_datetime(covid_df['Week Start']).dt.month

covid_df.head()

,ZIP,Week Number,Week Start,Week End,Cases - Weekly,Cases - Cumulative,Case Rate - Weekly,Case Rate - Cumulative,Tests - Weekly,Tests - Cumulative,...,Percent Tested Positive - Cumulative,Deaths - Weekly,Deaths - Cumulative,Death Rate - Weekly,Death Rate - Cumulative,Population,Row ID,ZIP Code Location,Year,Month
0,60612,15,4/5/2020,4/11/2020,69.0,202.0,201.0,588.7,284.0,803,...,0.3,3,7,8.7,20.4,34311,60612-2020-15,POINT (-87.687011 41.88004),2020,4
1,60612,16,4/12/2020,4/18/2020,66.0,268.0,192.0,781.1,286.0,1089,...,0.3,3,10,8.7,29.1,34311,60612-2020-16,POINT (-87.687011 41.88004),2020,4
2,60604,4,1/23/2022,1/29/2022,8.0,322.0,1023.0,41176.5,246.0,11461,...,0.0,0,0,0.0,0.0,782,60604-2022-4,POINT (-87.629029 41.878153),2022,1
3,60615,24,6/7/2020,6/13/2020,12.0,377.0,29.0,907.1,538.0,3986,...,0.1,0,15,0.0,36.1,41563,60615-2020-24,POINT (-87.602725 41.801993),2020,6
4,60615,25,6/14/2020,6/20/2020,13.0,390.0,31.0,938.3,551.0,4537,...,0.1,1,16,2.4,38.5,41563,60615-2020-25,POINT (-87.602725 41.801993),2020,6


In [47]:
# covid_df[ covid_df['ZIP'] == '60601' and covid_df['Year'] == "2020" ]
# covid_df.head()

In [40]:
covid_grouped = covid_df.groupby(['Year', 'Month', 'ZIP'])
# covid_grouped = covid_df.groupby(['ZIP'])
test = covid_grouped['Cases - Weekly'].sum()
# test.head()
test
# covid_df.columns

Year  Month  ZIP    
2020  3      60601        24.0
             60602         0.0
             60603         0.0
             60604         0.0
             60605        49.0
                         ...  
2022  1      60661       333.0
             60666         1.0
             60707       492.0
             60827        86.0
             Unknown    2275.0
Name: Cases - Weekly, Length: 1380, dtype: float64

In [23]:
len(mega_merge)
# len(covid_df)

48500

## Make sidewalk cafe permit data